In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/graph_train.csv', index_col=0)
y = pd.cut(df['edge_cross_norm'], bins=[-1,-1e-10,1]).cat.codes.to_numpy()
df = df.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg'],axis=1)
list_columns = list(df)

for col in list_columns:
    print(col)
    if col!= 'edge_cross_norm' and col!= 'is_bridge':
        df[col] = pd.qcut(df[col],q=5)

X = df.to_numpy()
df.head()


edge_betweenness
stress
is_bridge
diff_stress
diff_cross
diff_edgelength
exp_factor_norm
sum_neighbour_deg_norm
max_neighbour_deg_norm
grad_diff


,edge_betweenness,stress,is_bridge,diff_stress,diff_cross,diff_edgelength,exp_factor_norm,sum_neighbour_deg_norm,max_neighbour_deg_norm,grad_diff
0,"(0.0451, 0.0702]","(0.0818, 0.684]",False,"(0.357, 1.41]","(-2.0, 0.0]","(-0.00824, -0.00378]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
1,"(-0.000785, 0.0219]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
2,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
3,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(2.0, 39.0]","(-0.516, -0.00824]","(0.109, 0.607]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
4,"(0.0702, 0.545]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(-2.0, 0.0]","(0.00165, 0.266]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"


In [20]:
X_train,X_test, y_train, y_test = train_test_split(X,y)

In [21]:
y_train

array([1, 0, 0, ..., 0, 0, 0], dtype=int8)

In [22]:
# XGBoost prep
import numpy as np
df2 = pd.read_csv('../data/graph_train.csv', index_col=0)
df2['is_bridge'] = df2['is_bridge'].astype(float)
yn = (df2['edge_cross_norm']>0).to_numpy(dtype=np.int)
df2 = df2.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg'],axis=1)
Xn = df2.to_numpy(dtype=np.float64)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn,yn)

<ipython-input-22-5ee0f6ae93f1>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  yn = (df2['edge_cross_norm']>0).to_numpy(dtype=np.int)


In [23]:
yn[100]

0

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [36]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(Xn_train, yn_train, eval_metric = 'auc')
print(xgb)
yn_pred = xgb.predict(Xn_test)

np.sum(yn_pred == yn_test)/len(yn_pred)

/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)
[0 0 1 ... 0 0 1]


In [ ]:
yn_train

array([0, 0, 0, ..., 0, 1, 1])